#Pytorch Models in Production

## Installation of Torchserve



https://github.com/pytorch/serve/blob/master/docs/getting_started.md

## Creating Models using PyTorch

In [ ]:
import torch
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Importing Pretrained PyTorch Models


In [ ]:
from torchvision.models.alexnet import AlexNet

class ImageClassifier(AlexNet):
    def __init__(self):
        super(ImageClassifier, self).__init__()

## Example of a Handler

In [ ]:
"""
Module for image classification default handler
"""
import torch
import torch.nn.functional as F
from torchvision import transforms

from ts.vision_handler import VisionHandler
from ts.utils.util  import map_class_to_label


class ImageClassifier(VisionHandler):
    """
    ImageClassifier handler class. This handler takes an image
    and returns the name of object in that image.
    """

    topk = 5
    # These are the standard Imagenet dimensions
    # and statistics
    image_processing = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    def set_max_result_classes(self, topk):
        self.topk = topk

    def get_max_result_classes(self):
        return self.topk

    def postprocess(self, data):
        ps = F.softmax(data, dim=1)
        probs, classes = torch.topk(ps, self.topk, dim=1)
        probs = probs.tolist()
        classes = classes.tolist()
        return map_class_to_label(probs, self.mapping, classes)

## Saving Model using Scripting

In [ ]:
#scripted mode
from torchvision import models
import torch
model = models.alexnet(pretrained=True)
sm = torch.jit.script(model)
sm.save("alexnet.pt")

## Saving Model using Tracing

In [ ]:
#traced mode
from torchvision import models
import torch
model = models.alexnet(pretrained=True)
model.eval()
example_input = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("alexnet.pt")

## Torch Model Archiver

In [ ]:
torch-model-archiver --help

### Download Pretrained Image Classification Model like AlexNet, DenseNet

In [ ]:
wget https://download.pytorch.org/models/alexnet-owt-7be5be79.pth

### Combining all the files required for inference in single .mar file using torch-model-archiver

In [ ]:
torch-model-archiver --model-name alexnet --version 1.0 --model-file ./serve/examples/image_classifier/alexnet/model.py --serialized-file alexnet-owt-7be5be79.pth --handler image_classifier --extra-files ./serve/examples/image_classifier/index_to_name.json

### Create model store folder and add the .mar files to it for serving the models

In [ ]:
mkdir model_store
mv alexnet.mar model_store/

### Start the server

In [ ]:
torchserve --start --model-store model_store --models alexnet=alexnet.mar

### Example Inference - Image Classification

In [ ]:
curl http://127.0.0.1:8080/predictions/alexnet -T ./serve/examples/image_classifier/kitten.jpg

### Deployed Models Manager

In [ ]:
curl http://127.0.0.1:8081/models